Rare mutant ("superior over resident trat") invading a resident system (at fixation). System will always go to fixation before next mutation.
* Invasion fitness of $H_{mut}$ is given as $F_{inv} = F_{mut} - F_{res}$.
* Invasion possible if $F_{inv} > 0$, or $F_{mut} > F_{res}$.

**Example model:**  
* Attack rate a is evolving trait --> $a(x_{res})$  
* $x$ is a defense trait. Higher $x$ results in a lower attack rate.
  * Resulting in $a(x) = a_0(1 - \theta \cdot x)$
  * $\theta$ is the effectiveness of the defense
  * $x = 0$ is no defense

**Turing model**  
* Evolving trait is the sensitivity $k$

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import optimize as opt

In [ ]:
# var
var = []

# set parameters
S = 4.8 # Nutrient supply concentration
D = 0.3 # Dilution rate
N_h = 1.5 # half saturation constant for nutrient uptake
r_max = 0.7 # growth rate of autotroph
h = 0.53 # handling time
e = 0.33 # conversion efficiency of competitor
d_N = 1 # Dispersal rate of nutrients 
d_A = 0.001 # dispersal rate of autotrophs

# competitiveness 
a_1 = 1 # attack rate of competitor 1 
a_2 = 1 # attack rate of competitor 2 

# adaptability
k_1 = 0 # dispersal adaptability of competitor 1 
 #k_2 = 2 # dispersal adaptability of competitor 2 
        # 0 = random dispersal, 2 = adaptive dispersal

# dispersal speed 
d_Hmax1 = 0.01 # maximal dispersal rates of competitor 1  
d_Hmax2 = 0.01 # maximal dispersal rates of competitor 2  

# time series
t_end = 400
number_steps = 200
t = np.linspace(0,t_end,number_steps)

# initials:[N_a, N_b, A_a, A_b, H_1a, H_1b, H_2a, H_2b]
var0 = [2, 2.5, 2.5, 2, 0.08, 0.4, 0.08, 0.4]

In [ ]:
def PIPmodel(var, x_res):
    N_a = var[0]
    N_b = var[1]
    A_a = var[2]
    A_b = var[3]
    H_1a = var[4]
    H_1b = var[5]
    H_2a = var[6]
    H_2b = var[7]
    
    k_2 = k_2_0 * (1 - theta * x_res)

    # growth rate of autotrophs
    r_a = (r_max * N_a) / (N_h + N_a)
    r_b = (r_max * N_b) / (N_h + N_b)

    # growth rate of competitors
    g_1a = (a_1 * A_a) / (1 + a_1 * h * A_a)
    g_1b = (a_1 * A_b) / (1 + a_1 * h * A_b)
    g_2a = (a_2 * A_a) / (1 + a_2 * h * A_a)
    g_2b = (a_2 * A_b) / (1 + a_2 * h * A_b)

    # inflection points
    x_01 = D / (a_1 * (e - h * D))
    x_02 = D / (a_2 * (e - h * D))
    
    # dispersal rates of competitors
    d_H1a = d_Hmax1 / (1 + np.exp(k_1 * (A_a - x_01)))
    d_H1b = d_Hmax1 / (1 + np.exp(k_1 * (A_b - x_01)))
    d_H2a = d_Hmax2 / (1 + np.exp(k_2 * (A_a - x_02)))
    d_H2b = d_Hmax2 / (1 + np.exp(k_2 * (A_b - x_02)))

    # change of nutrients
    dN_a = D * (S - N_a) - r_a * A_a + d_N * (N_b - N_a)
    dN_b = D * (S - N_b) - r_b * A_b + d_N * (N_a - N_b)

    # change of autotrophs
    dA_a = r_a * A_a - ((g_1a * H_1a) + (g_2a * H_2a)) - D * A_a + d_A * (A_b - A_a)
    dA_b = r_b * A_b - ((g_1b * H_1b) + (g_2b * H_2b)) - D * A_b + d_A * (A_a - A_b)

    # change of competitors
    dH_1a = e * g_1a * H_1a - D * H_1a - d_H1a * H_1a + d_H1b * H_1b
    dH_1b = e * g_1b * H_1b - D * H_1b - d_H1b * H_1b + d_H1a * H_1a
    dH_2a = e * g_2a * H_2a - D * H_2a - d_H2a * H_2a + d_H2b * H_2b
    dH_2b = e * g_2b * H_2b - D * H_2b - d_H2b * H_2b + d_H2a * H_2a

    return(dN_a, dN_b, dA_a, dA_b, dH_1a, dH_1b, dH_2a, dH_2b)

* calculate the fixed point abundances for the entire trait range $($0 \leq x_{res} \leq 1$). 
* For simplicity copy from example:
  * $\theta = 0.999$
  * Start $k_2$ at $10$ and approach $0$.

In [12]:
def Calculate_FP(min, max, number_values):
    
    # resident trait values for which we want to calculate the fixed point:
    resident_values = np.linspace(min,max,number_values)
    
    # empty two-dimensional array that will store the ecological fixed points:
    FP_results = np.empty((2,number_values))
    
    # initial values for the root finding command: assume 0.1 for all populations
    initial_values = np.repeat(0.1,len(FP_results)) 
    
    # the calculations:
    for i in range(number_values):
        
        x_res = resident_values[i]
        FP = opt.fsolve(Ecological_Model,initial_values,args=(x_res))
        
        for j in range(len(FP_results)): # store the results returned by the root finder
            FP_results[j,i] = FP[j] 
        
    return [resident_values,FP_results]

0.010999000000000425